In [9]:
import sys
sys.path.append("../") # go to parent dir

from pprint import pprint
from pathlib import Path
import pyjson5
from dcf.utils import relative_error, average_dict_val
from dcf.ddm import diameter_calculator, diffusion_coefficient_calculator
import pandas as pd
import numpy as np

project_path = Path("..").resolve()

%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
ddm_data_path = project_path / "data" / "DDM" / "ddm_results.json5"
particle_data_path = project_path / "data" / "particle_data.json5"

with open(ddm_data_path) as f:
    ddm_data = pyjson5.load(f)

with open(particle_data_path) as f:
    particle_data = pyjson5.load(f)


In [11]:
# Normalise data
for exp_name, exp in ddm_data["experiments"].items():
    exp |= ddm_data["setups"][exp["setup"]]
    exp |= particle_data.get(exp["particle"], {})


In [12]:
# Convert to df
df = pd.DataFrame(ddm_data["experiments"]).transpose()
df

,particle,exp_brownian_diff_coeff,exp_brownian_diff_coeff_lower_95,exp_brownian_diff_coeff_upper_95,exp_freeexp_diff_coeff,exp_freeexp_diff_coeff_lower_95,exp_freeexp_diff_coeff_upper_95,exp_freeexp_alpha,exp_freeexp_alpha_lower_95,exp_freeexp_alpha_upper_95,concentration,num_frames,setup,objective,pix2mum,framerate,material_type,known_diameter,std_dev_diameter
1A05,A,0.256079,0.25161,0.260549,0.220893,0.212471,0.229316,1.844969,1.806618,1.88332,0.5,990,20x_1,20x,0.3008,100.0,SiO2,0.0,0.0
1A5,A,0.127455,0.123103,0.131807,0.189855,0.183172,0.196539,2.495567,2.460006,2.531127,5.0,989,20x_1,20x,0.3008,100.0,SiO2,0.0,0.0
1A5_2,A,0.285473,0.279319,0.291628,0.345363,0.337846,0.352879,2.265827,2.241944,2.28971,5.0,985,20x_1,20x,0.3008,100.0,SiO2,0.0,0.0
1B005,B,0.179529,0.177686,0.181372,0.177011,0.173668,0.180354,1.984946,1.967862,2.002029,0.05,950,20x_1,20x,0.3008,100.0,PS,0.0,None
1B005_2,B,0.182834,0.180607,0.185061,0.176739,0.172885,0.180594,1.963284,1.94301,1.983558,0.05,947,20x_1,20x,0.3008,100.0,PS,0.0,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2gradient01_2B-1D_3_highest,gradient,0.128639,0.127027,0.13025,0.12995,0.126328,0.133572,2.012335,1.982183,2.042486,0.1,989,20x_1,20x,0.3008,100.0,NaN,NaN,NaN
2gradient01_2B-1D_4_highest,gradient,0.092595,0.091674,0.093516,0.096822,0.095159,0.098485,2.068923,2.046792,2.091055,0.1,990,20x_1,20x,0.3008,100.0,NaN,NaN,NaN
2gradient01_2B-1D_5_highest,gradient,0.111622,0.110618,0.112627,0.111287,0.109274,0.1133,1.995927,1.974697,2.017157,0.1,991,20x_1,20x,0.3008,100.0,NaN,NaN,NaN
2gradient01_2B-1D_6_highest,gradient,0.138223,0.136809,0.139637,0.137111,0.134205,0.140018,1.989832,1.966364,2.0133,0.1,988,20x_1,20x,0.3008,100.0,NaN,NaN,NaN


In [13]:
# Compute data
df["temperature"] = 273 + 21
df["viscosity"] = 0.9775e-3
df["pix2mum"] = 0.234341

df["measurement_time"] = 10.0

# 6 degc measurements are done over 12 seconds, not 10
df.loc[df.index.str.endswith("_6C"), "measurement_time"] = 12.0
df.loc[df.index.str.endswith("_6C"), "temperature"] = 273 + 6
df.loc[df.index.str.endswith("_6C"), "viscosity"] = 1.4715e-3


df["framerate"] = df["num_frames"] / df["measurement_time"]

# Calculate experimental diameters
df["exp_diameter"] = df.apply(
    lambda row: diameter_calculator(
        diffusion_coefficient=row["exp_brownian_diff_coeff"] or np.NaN,
        viscosity=row["viscosity"],
        temperature=row["temperature"],
        pix2mum=row["pix2mum"],
        framerate=row["framerate"],
    ),
    axis="columns",
)

# Calculate theoretical diffusion coefficients
df["theory_brownian_diff_coeff"] = df.apply(
    lambda row: diffusion_coefficient_calculator(
        diameter=row.get("known_diameter", np.NaN),
        viscosity=row["viscosity"],
        temperature=row["temperature"],
    ),
    axis="columns",
)


# Calculate relative error
df["error"] = df.apply(
    lambda row: relative_error(
        known=row.get("known_diameter", np.NaN), measured=row["exp_diameter"]
    ),
    axis="columns",
)

df


,particle,exp_brownian_diff_coeff,exp_brownian_diff_coeff_lower_95,exp_brownian_diff_coeff_upper_95,exp_freeexp_diff_coeff,exp_freeexp_diff_coeff_lower_95,exp_freeexp_diff_coeff_upper_95,exp_freeexp_alpha,exp_freeexp_alpha_lower_95,exp_freeexp_alpha_upper_95,...,framerate,material_type,known_diameter,std_dev_diameter,temperature,viscosity,measurement_time,exp_diameter,theory_brownian_diff_coeff,error
1A05,A,0.256079,0.25161,0.260549,0.220893,0.212471,0.229316,1.844969,1.806618,1.88332,...,99.0,SiO2,0.0,0.0,294,0.000978,10.0,3.164733e-07,2.088145,0.499874
1A5,A,0.127455,0.123103,0.131807,0.189855,0.183172,0.196539,2.495567,2.460006,2.531127,...,98.9,SiO2,0.0,0.0,294,0.000978,10.0,6.364932e-07,2.088145,2.016555
1A5_2,A,0.285473,0.279319,0.291628,0.345363,0.337846,0.352879,2.265827,2.241944,2.28971,...,98.5,SiO2,0.0,0.0,294,0.000978,10.0,2.853284e-07,2.088145,0.352267
1B005,B,0.179529,0.177686,0.181372,0.177011,0.173668,0.180354,1.984946,1.967862,2.002029,...,95.0,PS,0.0,None,294,0.000978,10.0,4.704231e-07,0.957823,0.022659
1B005_2,B,0.182834,0.180607,0.185061,0.176739,0.172885,0.180594,1.963284,1.94301,1.983558,...,94.7,PS,0.0,None,294,0.000978,10.0,4.633828e-07,0.957823,0.007354
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2gradient01_2B-1D_3_highest,gradient,0.128639,0.127027,0.13025,0.12995,0.126328,0.133572,2.012335,1.982183,2.042486,...,98.9,NaN,NaN,NaN,294,0.000978,10.0,6.306364e-07,NaN,NaN
2gradient01_2B-1D_4_highest,gradient,0.092595,0.091674,0.093516,0.096822,0.095159,0.098485,2.068923,2.046792,2.091055,...,99.0,NaN,NaN,NaN,294,0.000978,10.0,8.752372e-07,NaN,NaN
2gradient01_2B-1D_5_highest,gradient,0.111622,0.110618,0.112627,0.111287,0.109274,0.1133,1.995927,1.974697,2.017157,...,99.1,NaN,NaN,NaN,294,0.000978,10.0,7.253065e-07,NaN,NaN
2gradient01_2B-1D_6_highest,gradient,0.138223,0.136809,0.139637,0.137111,0.134205,0.140018,1.989832,1.966364,2.0133,...,98.8,NaN,NaN,NaN,294,0.000978,10.0,5.875037e-07,NaN,NaN


In [14]:
dfe = df[["particle", "concentration", "error", "exp_diameter", "known_diameter", "temperature", "known_diameter", "num_frames", ]]

,error,particle,concentration,exp_diameter,temperature,known_diameter,num_frames
1A05,0.499874,A,0.5,3.164733e-07,294,0.0,990
1A5,2.016555,A,5.0,6.364932e-07,294,0.0,989
1A5_2,0.352267,A,5.0,2.853284e-07,294,0.0,985
1B005,0.022659,B,0.05,4.704231e-07,294,0.0,950
1B005_2,0.007354,B,0.05,4.633828e-07,294,0.0,947
...,...,...,...,...,...,...,...
2gradient01_2B-1D_3_highest,NaN,gradient,0.1,6.306364e-07,294,NaN,989
2gradient01_2B-1D_4_highest,NaN,gradient,0.1,8.752372e-07,294,NaN,990
2gradient01_2B-1D_5_highest,NaN,gradient,0.1,7.253065e-07,294,NaN,991
2gradient01_2B-1D_6_highest,NaN,gradient,0.1,5.875037e-07,294,NaN,988
